# Orders

✏️ **Exercise**

Today, we will investigate the **orders**, and their associated review score.

For that purpose, we will create one single data table containing **all our orders with some engineered statistics for them as additional columns.**

Our goal is to create the following DataFrame, which will come in very handy for our modeling phase

  - `order_id` (_str) the id of the order_
  - `wait_time` (_float) the number of days between order_date and delivered_date_
  - `expected_wait_time` (_float) the number of days between order_date and estimated_delivery_date_
  - `delay_vs_expected` (_float) if the actual delivery date is later than the estimated delivery date, returns the absolute number of days between the two dates, otherwise return 0_
  - `order_status` (_str) the status of the order_
  - `dim_is_five_star` (_int) 1 if the order received a five_star, 0 otherwise_
  - `dim_is_one_star` (_int) 1 if the order received a one_star, 0 otherwise_
  - `review_score`(_int) from 1 to 5_
  - `number_of_product` (_int) number of products that the order contains_
  - `number_of_sellers` (_int) number of sellers involved in the order_
  - `price` (_float) total price of the order paid by customer_
  - `freight_value` (_float) value of the freight paid by customer_
  - (Optional) `distance_customer_seller` (_float) the distance in km between customer and seller_
  
We also want to filter out "non-delivered" orders, unless explicitly specified

❓ **Your challenge**: 

- Implement each feature as a separate method within the `Order` class available at `olist/order.py`
- Then, create a method `get_training_data()` that returns the complete DataFrame.

Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Then copy the code into `order.py` once you are certain of your code logic
- Focus on the data manipulation logic now, we will analyse the dataset visually in the next challenges

<details>
    <summary>🔥 Notebook best practice (must read) </summary>

From now on, exploratory notebooks are going to get pretty long, and we strongly advise you to follow these notebook principles:
- Code your logic so that your Notebook can always be ran from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` or `_` for intermediary steps when you know you won't need them for long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyber nbextention `Collapsable Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you are between method brackets e.g. `group_by()` to get the docs! Repeat a few times to open it permanently

</details>





In [ ]:
# Auto reload imported module everytime a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

In [ ]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Import olist data
from olist.data import Olist
olist=Olist()
data=olist.get_data()
matching_table = olist.get_matching_table()

## Code `order.py`

In [ ]:
orders = data['orders'].copy() # good practice to be sure not to modify your `data` variable

### get_wait_time
Return a dataframe with [order_id, wait_time, expected_wait_time, delay_vs_expected, order_status]

Hints:
- Don't forget to convert dates from "string" type to "pandas.datetime' using [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
- Take time to understand what python [`datetime`](https://docs.python.org/3/library/datetime.html) objects are 

In [ ]:
# We give you the pseudo-code below for this first method:

# Inspect orders dataframe
# Filter dataframe on delivered orders
# handle datetime
# compute wait time
# compute expected wait time
# compute delay vs expected - carefully handles "negative" delays
# check new dataframe and copy code carefully to `olist/order.py`

In [ ]:
# Inspect orders dataframe
orders.head()

In [ ]:
orders.dtypes

In [ ]:
# handle datetime
dates = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date','order_delivered_customer_date',
         'order_estimated_delivery_date']

for dat in dates:
    orders[dat] = pd.to_datetime(orders[dat])

In [ ]:
orders.dtypes

In [ ]:

#wait_time (float) the number of days between order_date and delivered_date


orders['wait_time'] = orders['order_approved_at'].map(pd.Timestamp.to_julian_date) \
    - orders['order_delivered_customer_date'].map(pd.Timestamp.to_julian_date)


In [ ]:
# compute expected wait time
#expected_wait_time (float) the number of days between order_date and estimated_delivery_date

orders['expected_wait_time'] = orders['order_estimated_delivery_date'].map(pd.Timestamp.to_julian_date)\
    - orders['order_approved_at'].map(pd.Timestamp.to_julian_date) 
    



In [ ]:
#delay_vs_expected (float) if the actual delivery date is later than the estimated delivery date, /
#returns the absolute number of days between the two dates, otherwise return 0

orders['delay_vs_expected'] = orders['order_delivered_customer_date'].map(pd.Timestamp.to_julian_date) \
    - orders['order_estimated_delivery_date'].map(pd.Timestamp.to_julian_date)

# np.where(criteria, value if true, value if false)
orders['delay_vs_expected'] = np.where(orders['delay_vs_expected'] < 0, 0, orders['delay_vs_expected'])

#orders['delay_vs_expected'][orders['delay_vs_expected'] < 0] = 0


In [ ]:
# YOUR CODE HERE
orders[['order_id', 'wait_time', 'expected_wait_time', 'delay_vs_expected', 'order_status']]



In [ ]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
# Test it below
from olist.order import Order
Order().get_wait_time()

### get_review_score
     Returns a DataFrame with:
        order_id, dim_is_five_star, dim_is_one_star, review_score

In [ ]:
reviews = data['order_reviews'].copy()
reviews.head()

In [ ]:
#order_id, dim_is_five_star, dim_is_one_star, review_score
reviews[['order_id', 'dim_is_five_star', 'dim_is_one_star', 'review_score']].groupby('order_id').nunique()

In [ ]:
# Fill in the functions below, which you will have to apply "element-wise" to each Series in the next cell below
# So as to create the 2 new columns requested 

def dim_five_star(d):
    
    if d == 5:
        return 1
    else:
        return 0

def dim_one_star(d):
    if d == 1:
        return 1
    else:
        return 0

In [ ]:
reviews["dim_is_five_star"] = reviews["review_score"].map(dim_five_star) # --> Series([0, 1, 1, 0, 0, 1 ...])
reviews["dim_is_one_star"] = reviews["review_score"].map(dim_one_star) # --> Series([0, 1, 1, 0, 0, 1 ...])

returns reviews[['order_id', 'dim_is_five_star', 'dim_is_one_star', 'review_score']]

In [ ]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
# Test it below
from olist.order import Order
Order().get_review_score()

### Check your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('reviews',
    dim_five_star=dim_five_star(5),
    dim_not_five_star=dim_five_star(3),
    dim_one_star=dim_one_star(1),
    dim_not_one_star=dim_one_star(2)
)
result.write()
print(result.check())

### get_number_products:
     Returns a DataFrame with:
        order_id, number_of_products (total number of products per order)

In [ ]:
# YOUR CODE HERE

#order_id: order unique identifier
#order_item_id: sequential number identifying number of items included in the same order.
    
data['order_items'].head()

df = data['order_items'][['order_id', 'order_item_id']].groupby('order_id').sum()

df['order_item_id']



In [ ]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_number_products()

### get_number_sellers:
     Returns a DataFrame with:
        order_id, number_of_sellers (total number of unique sellers per order)

<details>
    <summary>Hint</summary>

`pd.Series.nunique()`
</details>

In [ ]:
# YOUR CODE HERE

data['order_items'].head()
#df.groupby('param')['group'].nunique()
a=data['order_items'][['order_id', 'seller_id']].groupby('order_id').nunique()

a.rename(columns={'seller_id': 'number_of_sellers'},
                           inplace=True)
a

In [ ]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_number_sellers()

### get_price_and_freight
     Returns a DataFrame with:
        order_id, price, freight_value

<details>
    <summary>Hint</summary>

`pd.Series.agg()` allows you to apply one transformation method per column of your groupby object
</details>

In [ ]:
# YOUR CODE HERE
data['order_items'][['order_id', 'price', 'freight_value']]

#OR

data['order_items'][['order_id', 'price', 'freight_value']].groupby('order_id').sum()

#data['order_items'].head()

In [ ]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_price_and_freight()

### get_distance_seller_customer (OPTIONAL - Try only after finishing today's challenges - Skip to next section)
[order_id, distance_seller_customer] (the distance in km between customer and seller)

💡Have a look at the `haversine_distance` formula we coded for you in the `olist.utils` module

In [ ]:
# YOUR CODE HERE

In [ ]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_distance_seller_customer()

# Test your newly coded module

❓ Time to code `get_training_data` making use of your previous coded methods.

In [ ]:
gwt_df = Order().get_wait_time()
grs_df = Order().get_review_score()

#grs_df = reviews[['order_id', 'dim_is_five_star', 'dim_is_one_star', 'review_score']].groupby('order_id').nunique()
final_df = gwt_df.merge(grs_df, on ='order_id')
gnp_df = Order().get_number_products()
final_df = final_df.merge(gnp_df, on ='order_id')

gns_df = Order().get_number_sellers()
final_df = final_df.merge(gns_df, on ='order_id')

gpf_df = Order().get_price_and_freight()
final_df = final_df.merge(gpf_df, on='order_id')

#final_df = final_df.set_index('order_id')

In [ ]:
final_df.isnull().sum()  ##(97007, 12)

In [ ]:
gwt_df.shape  ##(96,5)

In [ ]:
grs_df.shape   ##(,4)

In [ ]:
gnp_df.shape ##(,1)

In [ ]:
gns_df.shape

In [ ]:
gpf_df.shape


In [ ]:
data_df.shape

In [ ]:
data_df.isnull().sum()

In [ ]:
final_df

In [1]:
from olist.order import Order
from nbresult import ChallengeResult
data = Order().get_training_data()
result = ChallengeResult('training',
    shape=data.shape,
    columns=sorted(list(data.columns))
)
result.write()
print(result.check())

POrfavorrrrrrrrrrrrrrrr
============================= test session starts ==============================
platform linux -- Python 3.8.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /home/nandosoq/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/nandosoq/code/Nandosoq/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: anyio-3.2.1, dash-1.21.0
collecting ... collected 2 items

tests/test_training.py::TestTraining::test_training_data_columns PASSED  [ 50%]
tests/test_training.py::TestTraining::test_training_data_shape PASSED    [100%]

============================== 2 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/training.pickle

git commit -m 'Completed training step'

git push origin master


🏁 Congratulations! Commit and push your notebook before starting the next challenge.